In [16]:
import os, sys, copy

import numpy as np
import pandas as pd

from tabulate import tabulate
from fractions import Fraction

from helper import visualize_tabulation, all_slack_starting, dual_simplex_to_all_slack_starting, save_tabulation, load_tabulation

### Define the LP Problem

In [2]:
# no. of decision variables (N)
NO_OF_DECISION_VARS = 4

# no. of conditions 
NO_OF_CONDITIONS = 3

# --------------------------------- objective function ---------------------------------
# specify the objective function in the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>]
OBJ_FUNC = [4, 5, 9, 11]

# -------------------------------- type of optimization --------------------------------
OPT_TYPE = "MAX" # or "MIN" 

# ------------------------------------- conditions -------------------------------------
# specify each condition in a new row with the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>, <solution>]
conditions = [
    [  1,   1,   1,   1,  15],
    [  7,   5,   3,   2, 120],
    [  3,   5,  10,  15, 100],
]

In [3]:
# convert them to fractions
OBJ_FUNC = np.array(list(map(Fraction, OBJ_FUNC)))

for i in range(len(conditions)):
    conditions[i] = list(map(Fraction, conditions[i]))

conditions = np.array(conditions)

### Create the Variables and the Initial Tabulation

In [4]:
var_symbol_arr = []
basic_var_symbol_arr = []

OBJ_FUNC_VAR_SYMBOL = "P"
DECISION_VAR_SYMBOL = "X"
SLACK_VAR_SYMBOL    = "S"

# ========================= ALL THE VARIABLES =========================
var_symbol_arr.append(OBJ_FUNC_VAR_SYMBOL)

for i in range(NO_OF_DECISION_VARS):
    var_symbol_arr.append(DECISION_VAR_SYMBOL + str(i+1))

for i in range(NO_OF_CONDITIONS):
    var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))

print("variables:", var_symbol_arr)

# ========================== BASIC VARIABLES ==========================
basic_var_symbol_arr.append(OBJ_FUNC_VAR_SYMBOL)

for i in range(NO_OF_CONDITIONS):
    basic_var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))

print("basic variables:", basic_var_symbol_arr)

variables: ['P', 'X1', 'X2', 'X3', 'X4', 'S1', 'S2', 'S3']
basic variables: ['P', 'S1', 'S2', 'S3']


In [5]:
# verify the elements
OBJ_FUNC = np.array(OBJ_FUNC)
if len(OBJ_FUNC) != NO_OF_DECISION_VARS:
    raise Exception(f"the no. of coefficients in the objective function doesn't match tne number of decision variables...")

conditions = np.array(conditions)
if len(conditions) != NO_OF_CONDITIONS:
    raise Exception(f"the no. of specified conditions doesn't match 'NO_OF_CONDITIONS'...")
if len(conditions[0]) != NO_OF_DECISION_VARS + 1:
    raise Exception(f"the no. of coefficients in the conditions doesn't match tne number of decision variables...")

In [6]:
# find the size of a row
row_size = 1 + NO_OF_DECISION_VARS + NO_OF_CONDITIONS + 1

# find the column size
col_size = 1 + NO_OF_CONDITIONS # no. of basic vars = 1 (for objective function) + no. of conditions

# construct the initial tabulation
obj_func_row = np.concatenate(([1], -OBJ_FUNC, np.zeros([NO_OF_CONDITIONS]), [0]))
initial_tabulation = obj_func_row.reshape((1, -1))

for i in range(1, col_size):

    # condition rows
    temp = np.zeros([NO_OF_CONDITIONS])
    temp[i-1] = 1
    row = np.concatenate(([0], conditions[i-1][ :-1], temp, conditions[i-1][-1: ])).reshape((1, -1))

    initial_tabulation = np.concatenate((initial_tabulation, row), axis=0)

# convert all the elements into fractions
for i in range(initial_tabulation.shape[0]):
    initial_tabulation[i] = np.array(list(map(Fraction, initial_tabulation[i])))

visualize_tabulation(initial_tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr)

┌────┬─────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬───────┐
│    │   P │   X1 │   X2 │   X3 │   X4 │   S1 │   S2 │   S3 │   sol │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ P  │   1 │   -4 │   -5 │   -9 │  -11 │    0 │    0 │    0 │     0 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ S1 │   0 │    1 │    1 │    1 │    1 │    1 │    0 │    0 │    15 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ S2 │   0 │    7 │    5 │    3 │    2 │    0 │    1 │    0 │   120 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ S3 │   0 │    3 │    5 │   10 │   15 │    0 │    0 │    1 │   100 │
└────┴─────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴───────┘


### Perform the All-Slack Starting Method

In [7]:
# obtain the optimal tabulation
optimal_tabulation, basic_var_symbol_arr = all_slack_starting(initial_tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr, opt_type=OPT_TYPE)

==================== iteration no.: 1 ====================
entering var: X4
pivot col: [Fraction(-11, 1) Fraction(1, 1) Fraction(2, 1) Fraction(15, 1)]
solution : [Fraction(0, 1) Fraction(15, 1) Fraction(120, 1) Fraction(100, 1)]
ratio col: [-0.         15.         60.          6.66666667]
leaving var : S3
pivot row: [Fraction(0, 1) Fraction(3, 1) Fraction(5, 1) Fraction(10, 1)
 Fraction(15, 1) Fraction(0, 1) Fraction(0, 1) Fraction(1, 1)
 Fraction(100, 1)]
new basic variables: ['P', 'S1', 'S2', 'X4']
┌────┬─────┬──────┬───────────┬───────────┬──────┬──────┬──────┬────────────┬───────────┐
│    │   P │   X1 │        X2 │        X3 │   X4 │   S1 │   S2 │         S3 │       sol │
├────┼─────┼──────┼───────────┼───────────┼──────┼──────┼──────┼────────────┼───────────┤
│ P  │   1 │ -1.8 │ -1.33333  │ -1.66667  │    0 │    0 │    0 │  0.733333  │  73.3333  │
├────┼─────┼──────┼───────────┼───────────┼──────┼──────┼──────┼────────────┼───────────┤
│ S1 │   0 │  0.8 │  0.666667 │  0.333333 │

### Adding a New Constraint

In [7]:
# save the optimal tabulation
save_tabulation(optimal_tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr, file_name="opt_tabulation")

In [14]:
# load the modified tabulation
modified_tabulation, var_symbol_arr_, basic_var_symnbol_arr_ = load_tabulation(file_name="opt_tabulation")

In [15]:
visualize_tabulation(modified_tabulation, var_symbol_arr_, basic_var_symnbol_arr_)

┌────┬─────┬──────┬─────────┬──────┬─────────┬─────────┬──────┬─────────┬──────┬─────────┐
│    │   P │   X1 │      X2 │   X3 │      X4 │      S1 │   S2 │      S3 │   S4 │     sol │
├────┼─────┼──────┼─────────┼──────┼─────────┼─────────┼──────┼─────────┼──────┼─────────┤
│ P  │   1 │    0 │  0.4286 │    0 │  1.5714 │  1.8571 │    0 │  0.7143 │    0 │ 99.2857 │
├────┼─────┼──────┼─────────┼──────┼─────────┼─────────┼──────┼─────────┼──────┼─────────┤
│ X1 │   0 │    1 │  0.7143 │    0 │ -0.7143 │  1.4286 │    0 │ -0.1429 │    0 │  7.1429 │
├────┼─────┼──────┼─────────┼──────┼─────────┼─────────┼──────┼─────────┼──────┼─────────┤
│ S2 │   0 │    0 │ -0.8571 │    0 │  1.8571 │ -8.7143 │    1 │  0.5714 │    0 │ 46.4286 │
├────┼─────┼──────┼─────────┼──────┼─────────┼─────────┼──────┼─────────┼──────┼─────────┤
│ X3 │   0 │    0 │  0.2857 │    1 │  1.7143 │ -0.4286 │    0 │  0.1429 │    0 │  7.8571 │
├────┼─────┼──────┼─────────┼──────┼─────────┼─────────┼──────┼─────────┼──────┼─────────┤

In [16]:
modified_tabulation[4] = modified_tabulation[4] - 3*modified_tabulation[1] - 7*modified_tabulation[3]

In [17]:
visualize_tabulation(modified_tabulation, var_symbol_arr_, basic_var_symnbol_arr_)

┌────┬─────┬──────┬─────────┬──────┬─────────┬─────────┬──────┬─────────┬──────┬─────────┐
│    │   P │   X1 │      X2 │   X3 │      X4 │      S1 │   S2 │      S3 │   S4 │     sol │
├────┼─────┼──────┼─────────┼──────┼─────────┼─────────┼──────┼─────────┼──────┼─────────┤
│ P  │   1 │    0 │  0.4286 │    0 │  1.5714 │  1.8571 │    0 │  0.7143 │    0 │ 99.2857 │
├────┼─────┼──────┼─────────┼──────┼─────────┼─────────┼──────┼─────────┼──────┼─────────┤
│ X1 │   0 │    1 │  0.7143 │    0 │ -0.7143 │  1.4286 │    0 │ -0.1429 │    0 │  7.1429 │
├────┼─────┼──────┼─────────┼──────┼─────────┼─────────┼──────┼─────────┼──────┼─────────┤
│ S2 │   0 │    0 │ -0.8571 │    0 │  1.8571 │ -8.7143 │    1 │  0.5714 │    0 │ 46.4286 │
├────┼─────┼──────┼─────────┼──────┼─────────┼─────────┼──────┼─────────┼──────┼─────────┤
│ X3 │   0 │    0 │  0.2857 │    1 │  1.7143 │ -0.4286 │    0 │  0.1429 │    0 │  7.8571 │
├────┼─────┼──────┼─────────┼──────┼─────────┼─────────┼──────┼─────────┼──────┼─────────┤

In [20]:
modified_tabulation_1, basic_var_symbol_arr_1 = dual_simplex_to_all_slack_starting(modified_tabulation, all_vars=var_symbol_arr_, basic_vars=basic_var_symnbol_arr_)

====================DUAL SIMPLEX - iteration no.: 1====================
leaving variale : S4
ratio row       : [        inf         nan  3.                 nan  0.32352941  1.44444444
         nan  1.25        0.         19.85714286]
entering variale: X4
┌────┬─────┬──────┬─────────┬──────┬──────┬─────────┬──────┬─────────┬─────────┬─────────┐
│    │   P │   X1 │      X2 │   X3 │   X4 │      S1 │   S2 │      S3 │      S4 │     sol │
├────┼─────┼──────┼─────────┼──────┼──────┼─────────┼──────┼─────────┼─────────┼─────────┤
│ P  │   1 │    0 │  0.3824 │    0 │    0 │  1.4412 │    0 │  0.5294 │  0.3235 │ 97.6681 │
├────┼─────┼──────┼─────────┼──────┼──────┼─────────┼──────┼─────────┼─────────┼─────────┤
│ X1 │   0 │    1 │  0.7353 │    0 │    0 │  1.6176 │    0 │ -0.0588 │ -0.1471 │  7.8782 │
├────┼─────┼──────┼─────────┼──────┼──────┼─────────┼──────┼─────────┼─────────┼─────────┤
│ S2 │   0 │    0 │ -0.9118 │    0 │    0 │ -9.2059 │    1 │  0.3529 │  0.3824 │ 44.5168 │
├────┼─────┼─────

/home/akila/semester_8/OR/helper.py:150: RuntimeWarning: divide by zero encountered in divide
  ratio_row = np.abs(tabulation[0]) / np.abs(tabulation[pivot_row_idx])
/home/akila/semester_8/OR/helper.py:150: RuntimeWarning: invalid value encountered in divide
  ratio_row = np.abs(tabulation[0]) / np.abs(tabulation[pivot_row_idx])


### Introducing a New Product

In [9]:
# extract the contribution margin matrix
CMM = optimal_tabulation[:, [0, 5, 6, 7]]

In [11]:
# new product column in initial feasible tabulation
new_init_product_col = np.array([0, 1, 2, 3])

# new product column in the new final optimal tabulation
new_final_product_col = (CMM @ new_init_product_col).reshape(1, -1)[0]

# add the profit to the cost (substract 12)
new_final_product_col[0] -= 12

In [12]:
print(new_final_product_col)

[Fraction(-8, 1) Fraction(1, 1) Fraction(-5, 1) Fraction(0, 1)]


In [13]:
save_tabulation(optimal_tabulation, var_symbol_arr, basic_var_symbol_arr, file_name='opt_tabulation_1')

In [17]:
def load_tabulation(file_name):

    if not file_name.endswith('.csv'): file_name += '.csv'

    DF = pd.read_csv(file_name, index_col=0)  # Assuming the row labels are stored in the first column
    tabulation = DF.values
    for i in range(tabulation.shape[0]):
        tabulation[i] = np.array(list(map(Fraction, tabulation[i])))

    basic_vars = DF.index.tolist()
    all_vars = DF.columns.tolist()

    return tabulation, all_vars[ :-1], basic_vars

modified_tabulation_2, var_symbol_arr_2, basic_var_symbol_arr_2 = load_tabulation(file_name='opt_tabulation_1')

In [20]:
modified_tabulation_2

array([[Fraction(1, 1), Fraction(0, 1), Fraction(3, 7), Fraction(0, 1),
        Fraction(11, 7), Fraction(-8, 1), Fraction(13, 7),
        Fraction(0, 1), Fraction(5, 7), Fraction(695, 7)],
       [Fraction(0, 1), Fraction(1, 1), Fraction(5, 7), Fraction(0, 1),
        Fraction(-5, 7), Fraction(1, 1), Fraction(10, 7), Fraction(0, 1),
        Fraction(-1, 7), Fraction(50, 7)],
       [Fraction(0, 1), Fraction(0, 1), Fraction(-6, 7), Fraction(0, 1),
        Fraction(13, 7), Fraction(-5, 1), Fraction(-61, 7),
        Fraction(1, 1), Fraction(4, 7), Fraction(325, 7)],
       [Fraction(0, 1), Fraction(0, 1), Fraction(2, 7), Fraction(1, 1),
        Fraction(12, 7), Fraction(0, 1), Fraction(-3, 7), Fraction(0, 1),
        Fraction(1, 7), Fraction(55, 7)]], dtype=object)

In [19]:
# apply all-slack starting
all_slack_starting(modified_tabulation_2, var_symbol_arr_2, basic_var_symbol_arr_2, opt_type="MAX")

==================== iteration no.: 1 ====================
entering var: X5
pivot col: [Fraction(-8, 1) Fraction(1, 1) Fraction(-5, 1) Fraction(0, 1)]
solution : [Fraction(695, 7) Fraction(50, 7) Fraction(325, 7) Fraction(55, 7)]
ratio col: [-12.41071429   7.14285714  -9.28571429          inf]
leaving var : X1
pivot row: [Fraction(0, 1) Fraction(1, 1) Fraction(5, 7) Fraction(0, 1)
 Fraction(-5, 7) Fraction(1, 1) Fraction(10, 7) Fraction(0, 1)
 Fraction(-1, 7) Fraction(50, 7)]
new basic variables: ['P', 'X5', 'S2', 'X3']
┌────┬─────┬──────┬──────────┬──────┬───────────┬──────┬───────────┬──────┬───────────┬───────────┐
│    │   P │   X1 │       X2 │   X3 │        X4 │   X5 │        S1 │   S2 │        S3 │       sol │
├────┼─────┼──────┼──────────┼──────┼───────────┼──────┼───────────┼──────┼───────────┼───────────┤
│ P  │   1 │    8 │ 6.14286  │    0 │ -4.14286  │    0 │ 13.2857   │    0 │ -0.428571 │ 156.429   │
├────┼─────┼──────┼──────────┼──────┼───────────┼──────┼───────────┼──────

/home/akila/semester_8/OR/helper.py:69: RuntimeWarning: divide by zero encountered in divide
  print(f"solution : {solution}")


(array([[Fraction(1, 1), Fraction(8, 1), Fraction(7, 1), Fraction(3, 1),
         Fraction(1, 1), Fraction(0, 1), Fraction(12, 1), Fraction(0, 1),
         Fraction(0, 1), Fraction(180, 1)],
        [Fraction(0, 1), Fraction(1, 1), Fraction(1, 1), Fraction(1, 1),
         Fraction(1, 1), Fraction(1, 1), Fraction(1, 1), Fraction(0, 1),
         Fraction(0, 1), Fraction(15, 1)],
        [Fraction(0, 1), Fraction(5, 1), Fraction(3, 1), Fraction(1, 1),
         Fraction(0, 1), Fraction(0, 1), Fraction(-2, 1), Fraction(1, 1),
         Fraction(0, 1), Fraction(90, 1)],
        [Fraction(0, 1), Fraction(0, 1), Fraction(2, 1), Fraction(7, 1),
         Fraction(12, 1), Fraction(0, 1), Fraction(-3, 1), Fraction(0, 1),
         Fraction(1, 1), Fraction(55, 1)]], dtype=object),
 ['P', 'X5', 'S2', 'S3'])